In [1]:
import openai

client = openai.OpenAI()
messages = []



In [2]:
def call_ai_model(messages):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

In [3]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    else:
        messages.append({"role": "user", "content": user_input})
        print(f"User: {user_input}")
        response = call_ai_model(messages)
        messages.append({"role": "assistant", "content": response})
        print(response)

User: my name is nico
Nice to meet you, Nico! How can I assist you today?
User: what is my name
Your name is Nico. How can I help you today?
User: quir
It seems like you might be looking for a definition or context for the word "quir." It could be a misspelling of "quirk," which means a peculiar behavior or characteristic. If you meant something else or want to elaborate, please let me know!
User: quit
It looks like you might be expressing a desire to stop or end something. If you need help or support regarding a specific topic, or if you meant something else, feel free to elaborate!
